<a href="https://colab.research.google.com/github/nhsmith85/movie_ner/blob/main/MIT_Movie_1_data_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MIT Movie Dataset


In [ ]:
import pandas as pd
import numpy as np

# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

#os.chdir('/content/drive/My Drive/Colab Notebooks/DAAN888/data')
os.chdir('/content/drive/My Drive/DAAN888/data')
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1tuCnSXHVzIUUyCjMTEqmKbY7JxKZ-5eU/DAAN888/data'

### Retrieve Data from Web

In [ ]:
!wget https://groups.csail.mit.edu/sls/downloads/movie/engtrain.bio

In [ ]:
!wget https://groups.csail.mit.edu/sls/downloads/movie/engtest.bio

--2020-10-15 01:41:09--  https://groups.csail.mit.edu/sls/downloads/movie/engtest.bio
Resolving groups.csail.mit.edu (groups.csail.mit.edu)... 128.30.2.44
Connecting to groups.csail.mit.edu (groups.csail.mit.edu)|128.30.2.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252636 (247K)
Saving to: ‘engtest.bio.1’

engtest.bio.1       100%[===================>] 246.71K   749KB/s    in 0.3s    

2020-10-15 01:41:10 (749 KB/s) - ‘engtest.bio.1’ saved [252636/252636]



In [ ]:
!head -10 engtrain.bio

O	what
O	movies
O	star
B-ACTOR	bruce
I-ACTOR	willis

O	show
O	me
O	films
O	with


In [ ]:
!head -10 engtest.bio

O	are
O	there
O	any
O	good
B-GENRE	romantic
I-GENRE	comedies
O	out
B-YEAR	right
I-YEAR	now



### Tokenize the docs in text file


In [ ]:
from pathlib import Path

p = Path('engtrain.bio')

raw = p.read_text().strip()

raw[0:100]

'O\twhat\nO\tmovies\nO\tstar\nB-ACTOR\tbruce\nI-ACTOR\twillis\n\nO\tshow\nO\tme\nO\tfilms\nO\twith\nB-ACTOR\tdrew\nI-ACTOR'

In [ ]:
import re 

# find how many document breaks there are first doc doesn't need a line break
for p in Path('engtrain.bio'), Path('engtest.bio'):

  raw = p.read_text().strip()
  rx = r'\n\n'

  breaks = re.findall(rx, raw)
  print(p, len(breaks) )

engtrain.bio 9774
engtest.bio 2442


In [ ]:
def read_bio_files(filepath):
  '''this function will break up the text file into each sentence using the
  extra line \n as a breakpoint between docs and return a list of the individual
  words and a separate list of their corresponding labels '''

  text = Path(filepath).read_text().strip()
  each_doc = re.split(r'\n\t?\n', text)

  tokenized_docs = []
  labels = []

  for doc in each_doc:
    tokens = []
    tags = []
    for line in doc.split('\n'):
      tag, word = line.split('\t')
      tags.append(tag)
      tokens.append(word)

    tokenized_docs.append(tokens)
    labels.append(tags)

  return tokenized_docs, labels

In [ ]:
train_tokens, train_labels = read_bio_files('engtrain.bio')
test_tokens, test_labels = read_bio_files('engtest.bio')

In [ ]:
len(train_tokens), len(train_labels), len(test_tokens), len(test_labels)

(9775, 9775, 2443, 2443)

In [ ]:
len(train_tokens), len(train_labels), len(test_tokens), len(test_labels)

(9775, 9775, 2443, 2443)

In [ ]:
list(zip(train_tokens[0], train_labels[0]))

[('what', 'O'),
 ('movies', 'O'),
 ('star', 'O'),
 ('bruce', 'B-ACTOR'),
 ('willis', 'I-ACTOR')]

In [ ]:
list(zip(test_tokens[0], test_labels[0]))

[('are', 'O'),
 ('there', 'O'),
 ('any', 'O'),
 ('good', 'O'),
 ('romantic', 'B-GENRE'),
 ('comedies', 'I-GENRE'),
 ('out', 'O'),
 ('right', 'B-YEAR'),
 ('now', 'I-YEAR')]

In [ ]:
dataset = {'train_tokens': train_tokens, 
           'train_labels': train_labels,
           'test_tokens': test_tokens,
           'test_labels': test_labels}

In [ ]:
import pickle

with open('mitmovie.pickle', 'wb') as handle:
    pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)